### C3D

In [ ]:
import globo

import os , numpy as np , time , tensorflow as tf
from sklearn.preprocessing import normalize

import c3d
from utils import get_video_clips

'''
    Iterates over all UCF Crime video folders (train_0 , train_1 , test)
    Divide each video into snippets of 16 frames 
    Process each snippet trough c3dsports1m model
    #l2norm the features
    Saves a .npy file per video
'''

feature_extractor = c3d.c3d_feature_extractor()


@tf.function
def predict(x): return feature_extractor(x, training=False)

def l2norm(x): return x / np.linalg.norm(x, ord=2, axis=-1, keepdims=True)  


for type, dir in globo.UCFCRIME_VPATHS_LISTS.items():

    vpaths = list(open(dir))
    fpath = globo.UCFCRIME_C3D_FPATHS[type]

    print(f'\n****************\n\nProcessing {type} @ {dir}\n\n{len(vpaths)} vids\nsaving into{fpath}')
    
    for i, vpath in enumerate(vpaths):
        t = time.time()
        
        vpath = vpath.strip("\n")
        fn = os.path.splitext(os.path.basename(vpath))[0]
        print(f'\n{i} {fn}.mp4')
            
        out_npy = os.path.join(fpath, fn + ".npy")
        #if os.path.exists(out_npy):
        #    print(out_npy,"already created")
        #    continue
            
        clips, frames = get_video_clips(vpath) ## divide in2 snippets of 16
        
        features = []
        for clip in clips:
            prep_clip = c3d.preprocess_input(np.array(clip))
            #print(f'\tPREP {np.shape(prep_clip)}\n\t( {np.shape(prep_clip)[0]} frames , {np.shape(prep_clip)[1:]} resolution )',prep_clip.dtype)

            feature = predict(prep_clip)
            #print(f'\tFEAT {np.shape(feature)} ,{feature.dtype} , {isinstance(feature, np.ndarray)}')
            
            features.append(feature)
            
        features = np.concatenate(features)
        print(f'\tFEATED ( {np.shape(features)[0]} timesteps , {np.shape(features)[1]} features ) {features.dtype} , numpy ? {isinstance(features, np.ndarray)} , tf ? {isinstance(features, tf.Tensor)}')
        
        #features2 = normalize(features, axis=1)
        #features = l2norm(features)
        #print("\tL2NORM same ?",np.allclose(features,features2))
        
        #np.save(out_npy, features1)
        print(f'\n\tsaved @ {out_npy}')
        
        tt = time.time()
        print(f'\tin @ {int(tt-t)} seconds')
        
        break
        

### I3D

In [11]:
import globo
import numpy as np , cv2
import tensorflow as tf
import i3d_horse
from utils import gerador_clips

### VSWIN

In [ ]:
import globo , os
import numpy as np
import tensorflow as tf
from utils import *
from sklearn.preprocessing import normalize


'''
    Iterates over all UCF Crime video folders (train_0 , train_1 , test)
    Divide in each video into snippets of 16 frames 
    Process each snippet trough c3dsports1m model
    Saves a .npy file per video
'''

In [ ]:
def normalize_input(batch_clips):
    mean = np.array([0.485 * 255, 0.456 * 255, 0.406 * 255], dtype=np.float32)
    std = np.array([0.229 * 255, 0.224 * 255, 0.225 * 255], dtype=np.float32)
    return (batch_clips - mean) / std

In [ ]:
model = tf.saved_model.load(globo.VSWIN)

In [ ]:
shape_of_input = [1,3,32,224,224]
video = tf.random.normal(shape_of_input)
features= model(video)[0].numpy()
features.shape

In [ ]:
for type, dir in globo.UCFCRIME_VPATHS_LISTS.items():

    vpaths = list(open(dir))
    
    print(f'\n****************\n\nProcessing {type} @ {dir}\n\n{len(vpaths)} vids')
    
    features_list = []
    for i, vpath in enumerate(vpaths):
        
        vpath = vpath.strip('\n')
        fn = os.path.splitext(os.path.basename(vpath))[0]
        print(f'\n{i} {fn}.mp4\n')

        ## divide in2 snippets of 32
        clips, frames = get_video_clips(vpath , clip_length = 32 , resolution = 224)
        print(np.shape(clips) , clips.dtype)
        
        
        features = []
        for clip in clips:  
            clip_prep = normalize_input(clip)
            clip_prep_tf = tf.image.per_image_standardization(clip)
            print(np.shape(clip_prep),np.shape(clip_prep_tf))
            print(np.allclose(clip_prep , clip_prep_tf))
            
            clip_prep_tf = np.expand_dims(clip_prep_tf, axis=0)
            clip_prep_tf = np.transpose(clip_prep_tf, (0, 4, 1, 2, 3))
            print(np.shape(clip_prep_tf))
            
            feature = model(clip_prep_tf)[0].numpy()
            print(np.shape(feature))
            features.append(feature)
            
        print(np.shape(features))
        
        #features = normalize(features, axis=1)
        break

    # Save all the features for the current type as a single .npz file
    out_npz = os.path.join(globo.UCFCRIME_FEATC3D_BASE_DIR, type + ".npz")
    #np.savez(out_npz, *features_list)
    print(f'\n\tsaved {type} features @ {out_npz}')
